In [ ]:
import numpy as np
import pandas as pd
import requests, json
import sys


In [ ]:
pip install lxml

In [ ]:
file_name = 'loanphys_.csv' #input()
#'loanphys_.csv'
try:
  csvframe = pd.read_csv(file_name, delimiter=";", decimal=",")
except IOError:
  sys.exit("File doesn't exist")

In [ ]:
#досоздать строку в csvframe с датой 31.07.2020 для id = 62909499
copy = csvframe.iloc[1432].copy()
csvframe.loc[len(csvframe)] = copy

Функция, которая возвращает ссылку на страницу с сайта ЦБ с курсами валют на дату date:

In [ ]:
def get_link(date):
  old_link = 'https://cbr.ru/currency_base/daily/?UniDbQuery.Posted=True&UniDbQuery.To='
  new_link = old_link + date
  return new_link

Функция, которая возвращает таблицу с курсами валют за дату date:

In [ ]:
def currency_table(date):
  link = get_link(date)
  tables = pd.read_html(io=link, decimal=',', thousands='.')
  tables = tables[0]
  tables['Курс'] = tables['Курс'].astype(float)
  return tables


В таблице exchange_rate будут храниться курсы валют для всех дат из входного файла и для всех валют из входного файла

In [ ]:
dates = set(csvframe['REPORT_DATE'])
pattern_array = [1.0] * len(dates)
currencies = set(csvframe['CURRENCY_CODE'])

try:
  exchange_rate = pd.DataFrame(sorted(pd.to_datetime(list(dates), format = "%d.%m.%Y")), columns = ["Дата"]) #табличка с курсом валют по датам из csvframe
except:
  sys.exit("Данные столбца'REPORT_DATE' не являются датой")

exchange_rate.set_index("Дата", inplace=True)
for cur in currencies:
  exchange_rate[cur] = pattern_array

for date in dates:
  exchange = currency_table(date)
  exchange.set_index("Цифр. код", inplace=True)
  for cur in currencies:
    if (cur == 810):
      continue
    try:
      exchange_rate.loc[pd.to_datetime(date)][cur] = exchange.loc[cur]['Курс']
    except IndexError:
      sys.exit("Не существует такой валюты")

exchange_rate


,840,978,810
Дата,,,
2020-03-31,77.7325,85.7389,1.0
2020-04-30,73.6894,80.0488,1.0
2020-05-31,70.7520,78.5489,1.0
2020-06-30,69.9513,78.6812,1.0
2020-07-31,73.3633,86.2532,1.0
2020-08-31,74.6382,88.7448,1.0


In [ ]:
csvframe['REPORT_DATE'] = pd.to_datetime(csvframe['REPORT_DATE'])
csvframe = csvframe.sort_values (by=['DEAL_ID', 'REPORT_DATE'])
try:
  csvframe['DEBT_NM_AMT'] = csvframe['DEBT_NM_AMT'].astype(float, errors='raise')
except:
  sys.exit("Данные столбца 'DEBT_NM_AMT' не представимы в виде вещественного числа")
try:
  csvframe['PROVISION_RATE'] = csvframe['PROVISION_RATE'].astype(float, errors='raise')
except:
  sys.exit("Данные столбца 'PROVISION_RATE' не представимы в виде вещественного числа")
try:
  csvframe['PROVISION_DEBT_AMT'] = csvframe['PROVISION_DEBT_AMT'].astype(float, errors='raise')
except:
  sys.exit("Данные столбца 'PROVISION_DEBT_AMT' не представимы в виде вещественного числа")
csvframe = csvframe.fillna(0)
csvframe.index = list(range(0, len(csvframe), 1))
csvframe.head(12)


,REPORT_DATE,DEAL_ID,DEAL_PARENT_ID,DEAL_TYPE,BALACC_DEBT_CODE,BALACC_OVERDUE_CODE,PRODUCT_NAME,CURRENCY_CODE,BEGIN_DATE,END_FIRST_DATE,END_PLAN_DATE,TERM_OVERDUEDEBT_DAY_CNT,TERM_OVERDUEINTEREST_DAY_CNT,DEAL_AMT,DEBT_NM_AMT,DEBT_RUR_AMT,OVERDUE_NM_AMT,OVERDUE_RUR_AMT,INTEREST_RATE,INTEREST_NM_AMT,INTEREST_RUR_AMT,OVERDUEINTEREST_NM_AMT,OVERDUEINTEREST_RUR_AMT,RISKGROUP_CODE,PROVISION_RATE,PROVISION_DEBT_AMT,PROVISION_OVERDUE_AMT,PROVISION_INTEREST_AMT,PROVISION_OVERDUEINTEREST_AMT
0,2020-03-31,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,2924.0,2536.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
1,2020-04-30,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,2954.0,2566.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
2,2020-05-31,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,2985.0,2597.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
3,2020-06-30,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,3015.0,2627.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
4,2020-07-31,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,3046.0,2658.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
5,2020-08-31,6943416,0.0,0.0,0.0,45815.0,Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,3077.0,2689.0,41000.0,0.0,0.0,21463.40,21463.40,0.0,0.0,0.0,4523.44,4523.44,5,100.0,0.0,21463.40,0.0,4523.44
6,2020-03-31,6944919,0.0,0.0,0.0,45815.0,Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,2631.0,2569.0,88000.0,0.0,0.0,57469.55,57469.55,0.0,0.0,0.0,4273.67,4273.67,5,100.0,0.0,57469.55,0.0,4273.67
7,2020-04-30,6944919,0.0,0.0,0.0,45815.0,Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,2661.0,2599.0,88000.0,0.0,0.0,57469.55,57469.55,0.0,0.0,0.0,4273.67,4273.67,5,100.0,0.0,57469.55,0.0,4273.67
8,2020-05-31,6944919,0.0,0.0,0.0,45815.0,Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,2692.0,2630.0,88000.0,0.0,0.0,57469.55,57469.55,0.0,0.0,0.0,4273.67,4273.67,5,100.0,0.0,57469.55,0.0,4273.67
9,2020-06-30,6944919,0.0,0.0,0.0,45815.0,Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,2722.0,2660.0,88000.0,0.0,0.0,57469.55,57469.55,0.0,0.0,0.0,4273.67,4273.67,5,100.0,0.0,57469.55,0.0,4273.67


In [ ]:
csvframe.columns

Index(['REPORT_DATE', 'DEAL_ID', 'DEAL_PARENT_ID', 'DEAL_TYPE',
       'BALACC_DEBT_CODE', 'BALACC_OVERDUE_CODE', 'PRODUCT_NAME',
       'CURRENCY_CODE', 'BEGIN_DATE', 'END_FIRST_DATE', 'END_PLAN_DATE',
       'TERM_OVERDUEDEBT_DAY_CNT', 'TERM_OVERDUEINTEREST_DAY_CNT', 'DEAL_AMT',
       'DEBT_NM_AMT', 'DEBT_RUR_AMT', 'OVERDUE_NM_AMT', 'OVERDUE_RUR_AMT',
       'INTEREST_RATE', 'INTEREST_NM_AMT', 'INTEREST_RUR_AMT',
       'OVERDUEINTEREST_NM_AMT', 'OVERDUEINTEREST_RUR_AMT', 'RISKGROUP_CODE',
       'PROVISION_RATE', 'PROVISION_DEBT_AMT', 'PROVISION_OVERDUE_AMT',
       'PROVISION_INTEREST_AMT', 'PROVISION_OVERDUEINTEREST_AMT'],
      dtype='object')

Функция, которая возвращает курс за i-й день:

In [ ]:
def currency(i): 
  cur = csvframe['CURRENCY_CODE'][i]
  date = csvframe['REPORT_DATE'][i]
  return exchange_rate.loc[date][cur]

Функции, которые возвращают значения остатка, просрочки, ставки, суммы начисленных и просроченных процентов, суммы резерва под возможные потери ПОД, суммы резерва по процентам ОД, суммы резерва по процентам ПОД, суммы резерва под возможные потери ОД соответственно на  $i$-й день: 

In [ ]:
def debt(i):
  return (csvframe['DEBT_NM_AMT'][i])

def overdue_debt(i):
  return csvframe['OVERDUE_NM_AMT'][i]

def provision_rate(i):
  return (csvframe['PROVISION_RATE'][i] / 100)

def interest(i):
  return csvframe['INTEREST_NM_AMT'][i]

def overdue_interest(i):
  return csvframe['OVERDUEINTEREST_NM_AMT'][i]

def provision_overdue(i):
  return csvframe['PROVISION_OVERDUE_AMT'][i]

def provision_interest(i):
  return csvframe['PROVISION_INTEREST_AMT'][i]

def provision_overdueinterest(i):
  return csvframe['PROVISION_OVERDUEINTEREST_AMT'][i]

def provision_debt_amt(i):
  return csvframe['PROVISION_DEBT_AMT'][i]

Если элемент elem больше 0, то plus = elem, minus = 0, иначе minus = elem, plus = 0

In [ ]:
def sign_append(plus, minus, elem):
  if (elem > 0):
    plus = elem
    minus = 0
  else:
    plus = 0
    minus = elem

Функция, которая возвращает сумму резерва под возможные потери ОД, под возможные потери ПОД, по процентам ОД, по процентам ПОД

In [ ]:
def sum_provision(i):
  return provision_interest(i) + provision_overdueinterest(i) + provision_debt_amt(i) + provision_overdue(i)


Создадим шаблон таблицы с показателями формирования и восстановления резерва.
Названия столбцов:

$\underline{Формирование \; резерва:}$

Столбец 'provision_form_dA' -- выдача новых ссуд (основной долг) \\
Столбец 'provision_form_dR' -- увеличение ставки резервирования (основной долг) \\
Столбец 'provision_form_dC' -- изменение курса валюты (основной долг) \\
Столбец 'provision_form_dO' -- прочие причины (основной долг) \\


$\underline{Восстановление \; резерва:}$

Столбец 'provision_rec_dA' -- погашение  ссуд (основной долг) \\
Столбец 'provision_rec_dR' -- уменьшение ставки резервирования (основной долг) \\
Столбец 'provision_rec_dC' -- изменение курса валюты (основной долг) \\
Столбец 'provision_rec_dO' -- прочие причины (основной долг) \\


In [ ]:
pattern_array = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
data = dict()
sorted_dates = sorted(list(set((csvframe['REPORT_DATE']))))
for i in range(1, len(sorted_dates)):
  date = sorted_dates[i]
  data[date] = pattern_array
delta_provision = pd.DataFrame(data) #таблица для основного долга
delta_provision['parameter'] = ['sum_form', 'provision_form_dA', 'provision_form_dR', 'provision_form_dC', 'provision_form_dO', 
                      'sum_rec', 'provision_rec_dA', 'provision_rec_dR', 'provision_rec_dC', 'provision_rec_dO']

delta_provision.set_index("parameter", inplace=True)
delta_provision_all_id = {} #словарь с таблицами типа delta_provision для каждого DEAL_ID

delta_provision


,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
parameter,,,,,
sum_form,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dC,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dO,0.00000,0.00000,0.00000,0.00000,0.00000
sum_rec,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dC,0.00000,0.00000,0.00000,0.00000,0.00000


Заполним таблицы и посчитаем суммарное dA, dR, dC, dOther для всего портфеля

In [ ]:
sum_delta_provision = delta_provision.copy() #таблица с изменениями резерва для всего портфеля
sum_delta_provision.loc['dA_plus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dR_plus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dC_plus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dO_plus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dA_minus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dR_minus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dC_minus'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dO_minus'] = [0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
i = 0
while (i != len(csvframe)):
    j = 0
    temp_table = delta_provision.copy() #создаем табличку с показателями
    deal_id = csvframe['DEAL_ID'][i]
    while (j+i+1 < len(csvframe) and 
           csvframe['DEAL_ID'][j+i] == csvframe['DEAL_ID'][j+1+i]):
      debt_nm_amt1 = debt(i+j) + overdue_debt(i+j)
      debt_nm_amt2 = debt(i+j+1) + overdue_debt(i+j+1)
      c1 = currency(i+j)   
      c2 = currency(i+j+1)
      provision_rate1 = provision_rate(i+j)
      provision_rate2 = provision_rate(i+j+1)
      date = csvframe['REPORT_DATE'][i+j+1]
      d_a = (debt_nm_amt2 - debt_nm_amt1) * provision_rate1 * c1
      d_c = (c2 - c1) * provision_rate1 * debt_nm_amt2
      d_r = (provision_rate2 - provision_rate1) * debt_nm_amt2 * c2
      d_other = sum_provision(i+j+1) - sum_provision(i+j) - d_a - d_c - d_r
      j += 1
      
      if d_a > 0:
        sum_delta_provision[date]['dA_plus'] += d_a
        temp_table[date]['provision_form_dA'] += d_a
        temp_table[date]['sum_form'] += d_a
      else:
        sum_delta_provision[date]['dA_minus'] += d_a
        temp_table[date]['provision_rec_dA'] += d_a
        temp_table[date]['sum_rec'] += d_a
      if d_r > 0:
        sum_delta_provision[date]['dR_plus'] += d_r
        temp_table[date]['provision_form_dR'] += d_r
        temp_table[date]['sum_form'] += d_r
      else:
        sum_delta_provision[date]['dR_minus'] += d_r
        temp_table[date]['provision_rec_dR'] += d_r
        temp_table[date]['sum_rec'] += d_r
      if d_c > 0:
        sum_delta_provision[date]['dC_plus'] += d_c
        temp_table[date]['provision_form_dC'] += d_c
        temp_table[date]['sum_form'] += d_c
      else:
        sum_delta_provision[date]['dC_minus'] += d_c
        temp_table[date]['provision_rec_dC'] += d_c
        temp_table[date]['sum_rec'] += d_c
      if d_other > 0:
        sum_delta_provision[date]['dO_plus'] += d_other
        temp_table[date]['provision_form_dO'] += d_other
        temp_table[date]['sum_form'] += d_other
      else:
        sum_delta_provision[date]['dO_minus'] += d_other
        temp_table[date]['provision_rec_dO'] += d_other
        temp_table[date]['sum_rec'] += d_other
    delta_provision_all_id[deal_id] = temp_table #добавляем табличку с показателями в словарь
    i += (j+1)


Получили dA, dR, dC, dOther для портфеля на каждую из дат. Теперь составим таблицу:

In [ ]:
def count_parameter(sum_delta_provision, date):
  sum_delta_provision[date]['provision_form_dA'] = sum_delta_provision[date]['dA_plus']
  sum_delta_provision[date]['sum_form'] += sum_delta_provision[date]['dA_plus']
  sum_delta_provision[date]['provision_rec_dA'] = sum_delta_provision[date]['dA_minus']
  sum_delta_provision[date]['sum_rec'] += sum_delta_provision[date]['dA_minus']
  sum_delta_provision[date]['provision_form_dR'] = sum_delta_provision[date]['dR_plus']
  sum_delta_provision[date]['sum_form'] += sum_delta_provision[date]['dR_plus']
  sum_delta_provision[date]['provision_rec_dR'] = sum_delta_provision[date]['dR_minus']
  sum_delta_provision[date]['sum_rec'] += sum_delta_provision[date]['dR_minus']
  sum_delta_provision[date]['provision_form_dC'] = sum_delta_provision[date]['dC_plus']
  sum_delta_provision[date]['sum_form'] += sum_delta_provision[date]['dC_plus']
  sum_delta_provision[date]['provision_rec_dC'] = sum_delta_provision[date]['dC_minus']
  sum_delta_provision[date]['sum_rec'] += sum_delta_provision[date]['dC_minus']
  sum_delta_provision[date]['provision_form_dO'] += sum_delta_provision[date]['dO_plus']
  sum_delta_provision[date]['sum_form'] += sum_delta_provision[date]['dO_plus']
  sum_delta_provision[date]['provision_rec_dO'] = sum_delta_provision[date]['dO_minus']
  sum_delta_provision[date]['sum_rec'] += sum_delta_provision[date]['dO_minus']
  d_other_plus = sum_delta_provision[date]['dO_plus']

In [ ]:
count_parameter(sum_delta_provision, pd.to_datetime('2020-04-30'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-05-31'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-06-30'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-07-31'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-08-31'))

In [ ]:
sum_delta_provision.drop(["dA_plus", "dR_plus", "dC_plus", "dO_plus"], inplace = True)
sum_delta_provision.drop(["dA_minus", "dR_minus", "dC_minus", "dO_minus"], inplace = True)
pd.options.display.float_format ='{:.5f}'.format
sum_delta_provision

,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
parameter,,,,,
sum_form,11350439.59110,3661272.00556,6895531.23919,20406043.87802,7270466.25877
provision_form_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dR,9298608.38221,1528948.30650,4210347.68861,6884952.29201,1182157.82725
provision_form_dC,0.00000,0.00000,34653.65520,6207633.66841,2228724.60539
provision_form_dO,2051831.20889,2132323.69906,2650529.89538,7313457.91760,3859583.82613
sum_rec,-38740052.93110,-11825117.77556,-12872618.75919,-11895880.66802,-5893967.71877
provision_rec_dA,-10693451.36686,-3195096.61113,-5449336.07007,-9483671.10307,-4196794.28943
provision_rec_dR,-1942179.69617,-2452675.92405,-1817815.70315,-1883885.85731,-1473128.17258
provision_rec_dC,-6506948.83014,-4033822.02895,-991468.43313,0.00000,0.00000


In [ ]:
sum_delta_provision['Показатель'] = ['1. Формирование резерва, в том числе:',
                                    '1.1. выдача новых ссуд',
                                    '1.2. увеличение ставки резервирования',
                                    '1.3. изменение курса валюты',
                                    '1.4. прочие причины',
                                    '2. Восстановление резерва, в том числе:',
                                    '2.1. погашение ссуд',
                                    '2.2. уменьшение ставки резервирования',
                                    '2.3. изменение курса валюты',
                                    '2.4. прочие причины']

sum_delta_provision.set_index("Показатель", inplace=True)
sum_delta_provision


,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
Показатель,,,,,
"1. Формирование резерва, в том числе:",11350439.59110,3661272.00556,6895531.23919,20406043.87802,7270466.25877
1.1. выдача новых ссуд,0.00000,0.00000,0.00000,0.00000,0.00000
1.2. увеличение ставки резервирования,9298608.38221,1528948.30650,4210347.68861,6884952.29201,1182157.82725
1.3. изменение курса валюты,0.00000,0.00000,34653.65520,6207633.66841,2228724.60539
1.4. прочие причины,2051831.20889,2132323.69906,2650529.89538,7313457.91760,3859583.82613
"2. Восстановление резерва, в том числе:",-38740052.93110,-11825117.77556,-12872618.75919,-11895880.66802,-5893967.71877
2.1. погашение ссуд,-10693451.36686,-3195096.61113,-5449336.07007,-9483671.10307,-4196794.28943
2.2. уменьшение ставки резервирования,-1942179.69617,-2452675.92405,-1817815.70315,-1883885.85731,-1473128.17258
2.3. изменение курса валюты,-6506948.83014,-4033822.02895,-991468.43313,0.00000,0.00000
